In [1]:
import os
import pickle
import numpy 
import pandas as pd
#import cv2
import matplotlib.pyplot as plt
from PIL import Image
from pylab import *
from matplotlib import pyplot

from scipy. ndimage import filters
import pandas as pd
import os
from keras.models import Model, Sequential, save_model, load_model
import math
from sklearn.metrics import mean_squared_error

import random
from sklearn.preprocessing import MinMaxScaler
import h5py
import tensorflow as tf


Using TensorFlow backend.


In [2]:
import os 
import pickle
import numpy as np
import random

def preprocess(data_dirs, data_file,name):
    # all_ped_data would be a dictionary with mapping from each ped to their
    # trajectories given by matrix 3 x numPoints with each column
    # in the order x, y, frameId
    # Pedestrians from all datasets are combined
    # Dataset pedestrian indices are stored in dataset_indices
    all_ped_data={}
    dataset_indices=[]
    current_ped = 0
    # For each dataset
    for directory in data_dirs:
        # Define the path to its respective csv file
        #PETS2009-S2L1-mmundo.csv
        #'PETS09-S2L1.txt'
        file_path = os.path.join(directory,name )

        # Cargar datos desde el archivo csv
        # Los datos son una matriz 4x numTrajPoints
        
        
        data = np.genfromtxt(file_path, delimiter=',')
       
        
        # Obtenga el numero de peatones en el conjunto de datos actual
        uni=np.unique(data[:,1])    
        numPeds=np.size(np.unique(data[:,1]))
        
        print("peatones.............")
        print(numPeds)
       
        # Para cada peaton en el conjunto de datos
        for ped in range(1, numPeds+1):
            #Son los datos de la persona ped
            traj = data[ data[:, 1] == ped]
            #Esta como (x,y,frame_Id)
            traj = traj[:, [2,3,0]]
            
            #Esta como [[x,...],[y,...],[Frame_Id,..]]
            traj=[list(traj[:,0]),list(traj[:,1]),list(traj[:,2])]
            all_ped_data[current_ped + ped] = np.array(traj)

        # Current dataset done
        dataset_indices.append(current_ped+numPeds)
        current_ped += numPeds

    # Los datos completos son una tupla de todos los datos de peatones y los indices de datos del conjunto de datos.
    complete_data = (all_ped_data, dataset_indices)
    # Almacena los datos completos en el archivo pickle
    f = open(data_file, "wb")
    pickle.dump(complete_data, f, protocol=2)
    f.close()
    return complete_data

In [3]:
def load_preprocessed(data_file,seq_length_obs,batch_size):
    
    # cargar los datos
    f = open(data_file, "rb")
    raw_data = pickle.load(f)
    f.close()

    # Obtengo los peatones
    all_ped_data =raw_data[0]
 
    #all_ped_data es el vector {[[],[],[]],...,[[],[],[]]} tantos como pedestrian hayan
    # No usamos dataset_indices por ahora
    # dataset_indices = self.raw_data[1]

    # Construimos data con secuencias(o trayectorias) del largo than seq_length_obs
    data = []
    counter = 0

    # para cada peaton en data
    for ped in all_ped_data:
        
        #ped es un numero entero y va de 1 hasta la cantidad total de pedestrian
        # Extract his trajectory
        traj = all_ped_data[ped]
           
        #print(traj.shape)
        # If the length of the trajectory is greater than seq_length (+2 as we need both source and target data)
        #solo se toman las trajectorias de longitud mayor a seq_length+2
        if traj.shape[1] >= (seq_length_obs+1):
            # TODO: (Improve) Store only the (x,y) coordinates for now
            
            #print(traj[[0, 1], :].shape)
            data.append(traj[[0, 1], :].T)
            #print(traj[[0, 1], :].T)
            # Number of batches this datapoint is worth
            #print(traj.shape[1] )
            #print(int(traj.shape[1] / ((seq_length_obs+1))))
            counter += int(traj.shape[1] / ((seq_length_obs+1)))
        
    # Calculate the number of batches (each of batch_size) in the data
    #counter tiene la cantidad de bloques de 8 pasos
        
    num_batches = int(counter /batch_size)
    #cada bache tiene batch_size conjuntos donde cada conjunto tiene datos de length+2
    return data,num_batches

In [7]:

data_dirs = ['../data1/pets']
datasets=[0]
#data_dirs=['../data1/ucy/univ','../data1/ucy/zara/zara02','../data1/eth/hotel','../data1/ucy/zara/zara01']

#prueba=['../data1/eth/univ']
#datasets=[0,1,2,3]
used_data_dirs = [data_dirs[x] for x in datasets]
#Directorio de datos donde reside el archivo pickle preprocesado
data_dir = '../data1'

#Defina la ruta del archivo en el que deben almacenarse los datos.
data_file = os.path.join(data_dir, "datos_limpios_.cpkl")

name ='pixel_pos.csv'
# If the file doesn't exist already or if forcePreProcess is true
#se usa preprocesscsv para los csv y preprocess para el .txt

#print(used_data_dirs)
data = preprocess(used_data_dirs, data_file,name)
datos,numero = load_preprocessed(data_file,12,1)#los ultimos dos valores noimportan

peatones.............
19


In [12]:
len(datos[4])

220

In [ ]:
len(datos[7])

In [ ]:
def secuencia(seq_length_obs,data):  
    tamano=int(len(data))
    
    X,Y=[],[]
    for j in range(tamano):
        traj = data[j]
        lon = traj.shape[0]-seq_length_obs
    #lon=data.shape[0]-seq_length_obs-seq_length_pred
        for i in range(0,lon):
            a = traj[i:(i +seq_length_obs ), :]
            X.append(a)
            b = traj[i +seq_length_obs,:]
            Y.append(b)
    return np.array(X),np.array(Y)

## Visualización de los Datos

In [ ]:
# plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
import random
color_names = ["r","crimson" ,"g", "b","c","m","y","lightcoral", "peachpuff","grey","springgreen" ,"fuchsia","violet","teal","seagreen","lime","yellow","coral","aquamarine","hotpink"]

for i in range(len(datos)):
    cpu = random.choice(range(17))
    pintar = plt.plot(datos[i][:,0],datos[i][:,1],color=color_names[i])
plt.title(" Trayectorias completas de PETS-2009") 
plt.xlabel("Coordenada x")   
plt.ylabel("Coordenada y") 

#plt.savefig("trayectorias.jpg")
plt.show() 
"""

print(len(datos[3][:,0]))
print(len(datos[4][:,0]))

for i in range(4,7):
    cpu = random.choice(range(17))
    if(i==4):
        pintar = plt.plot(datos[i][165:180,0],datos[i][165:180,1],color=color_names[cpu])
    if(i==6): 
        #print(len(datos[6][:,1]))
        pintar = plt.plot(datos[i][15:30,0],datos[i][15:30,1],color=color_names[cpu])
        
plt.title(" Trayectorias completas de PETS-2009") 
plt.xlabel("Coordenada x")   
plt.ylabel("Coordenada y") 

#plt.savefig("trayectorias.jpg")
plt.show() 
"""

## Como se divide los conjuntos de datos

In [ ]:
longitud=len(datos)
ind = range(longitud)

lista = []
for i in range(0,longitud,4):
    lista.append(ind[i:i+4])
print(lista)

combinacion=[(0,1,2,3,4),(0,1,2,4,3),(0,1,3,4,2),(0,2,3,4,1),(1,2,3,4,0)]

def conjunto_datos(combinacion,lista,datos):
    conjunto_entrenamiento = []
    for i in range(len(combinacion)-1):
       # print(i)
        for j in lista[combinacion[i]]:
        #    print(j)
            conjunto_entrenamiento.append(datos[j])
    conjunto_prueba=[]
    for i in lista[combinacion[4]]:
        #print(shape(datos[i]))
        conjunto_prueba.append(datos[i])
    
    return conjunto_entrenamiento,conjunto_prueba

In [ ]:
train1,test1 = conjunto_datos(combinacion[0],lista,datos)
#vali1=[test1[0]]
train2,test2 = conjunto_datos(combinacion[1],lista,datos)
#vali2=[test2[0]]
train3,test3 = conjunto_datos(combinacion[2],lista,datos)
train4,test4 = conjunto_datos(combinacion[3],lista,datos)
train5,test5 = conjunto_datos(combinacion[4],lista,datos)


In [ ]:
length_obs = 8
trainX,trainY = secuencia(length_obs,train5)
#valiX,valiY = secuencia(length_obs,vali2)

In [ ]:
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1],trainX.shape[2]))
#valiX= numpy.reshape(valiX, (valiX.shape[0], valiX.shape[1],valiX.shape[2]))

In [ ]:
data_shape = trainX.shape[1:]
print(data_shape)
import numpy as np
np.shape(trainX)

## Para entrenar varios

In [ ]:
length_obs = 8
trainX,trainY = secuencia(length_obs,datos)

In [ ]:
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1],trainX.shape[2]))

In [ ]:
data_shape = trainX.shape[1:]
print(data_shape)
import numpy as np
np.shape(trainX)

## Diseño de la red

In [ ]:
from keras.models import Sequential
from keras import optimizers
#from keras.regularizers import l1, l2, l1_l2
from keras.layers import Dense
from keras.layers import LSTM
#from keras.layers import GRU
from keras.callbacks import EarlyStopping
from keras import initializers
from datetime import datetime
from keras.utils import plot_model



if 'model' in globals(): del model
model = None
model = Sequential()
model.add(LSTM(9, return_sequences=True, input_shape=data_shape, name='lstm1'))
model.add(LSTM(9, name='lstm2'))
model.add(Dense(2))

In [ ]:
model.summary()

In [ ]:
#validation_data=(vali_obs, vali_pred),
opt = optimizers.RMSprop(lr = 0.006, decay=1e-2)
model.compile(optimizer=opt, loss='logcosh',metrics=['mse'])
history= model.fit(trainX, trainY, epochs=100, batch_size=16, verbose=2)
#plot_model(model, show_shapes=True, to_file='modelo.png')

In [ ]:
history_dict= history.history
history_dict.keys()

In [ ]:
acc = history.history['mean_squared_error']
#val_acc = history.history['val_mean_squared_error']
loss = history.history['loss']
#val_loss = history.history['val_loss']

epochs = range(1, len(loss)+1)

# figure
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(epochs, loss, 'b', label='Training loss')
#plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, acc, 'r', label='Training mse')
#plt.plot(epochs, val_acc, 'g', label='Validation mse')
plt.title('Training mse')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
#model.save('lstm-xy1.h5')

In [ ]:
#model.save('lstm-xy2.h5')

In [ ]:
#model.save('lstm-xy3.h5')

In [ ]:
#model.save('lstm-xy4.h5')

In [ ]:
#model.save('lstm-xy5.h5')

In [ ]:
#model.save('lstm-xy1-simplicado.h5')

In [ ]:
#model.save('lstm-xy2-simplicado.h5')

In [ ]:
#model.save('lstm-xy3-simplicado.h5')

In [ ]:
#model.save('lstm-xy5-simplicado.h5')

In [ ]:
#model.save('lstm-xy-ucy-univ.h5')

In [ ]:
#model.save('lstm-xy-eth-hotel.h5')

In [ ]:
#model.save('lstm-xy-ucy-zara-zara01.h5')

In [ ]:
#model.save('lstm-xy-ucy-zara-zara02.h5')

In [ ]:
#model.save('lstm-xy-eth-univ.h5')

In [ ]:
modelxy = load_model('lstm-xy5-simplicado.h5')

In [ ]:
data_dirs=['../data1/ucy/univ']
datasets=[0]

used_data_dirs = [data_dirs[x] for x in datasets]
#Directorio de datos donde reside el archivo pickle preprocesado
data_dir = '../data1'

#Defina la ruta del archivo en el que deben almacenarse los datos.
data_file = os.path.join(data_dir, "datos_limpios_prueba.cpkl")

name ='pixel_pos.csv'
# If the file doesn't exist already or if forcePreProcess is true
#se usa preprocesscsv para los csv y preprocess para el .txt

#print(used_data_dirs)
data1 = preprocess(used_data_dirs, data_file,name)
datos_prueb,numero = load_preprocessed(data_file,12,1)#los ultimos dos valores noimportan

## Errores para medir las trayectorias predichas

In [ ]:
def error_promedio(predicted_traj, true_traj, observed_length):
    
    error = np.zeros(len(true_traj) - observed_length)
    # PARA CADA PUNTO EN LA TRAYECTORIA PREDICHA
    for i in range(observed_length, len(true_traj)):
        # The predicted position
        pred_pos = predicted_traj[i]
        # The true position
        true_pos = true_traj[i]

        # The euclidean distance is the error
        error[i-observed_length] = np.linalg.norm(true_pos - pred_pos)

    # Return the mean error
    return np.mean(error)


In [ ]:
def error_desplazamiento_final(predicted_traj, true_traj):
    tam = len(predicted_traj)
    return np.linalg.norm(predicted_traj[tam-1]-true_traj[tam-1])

In [ ]:
#esta funcion construye secuencias con solo la parte observada y secuencias completa
# es decir secuencia de longitud obs+pred

def secuencia_pred(seq_length_obs,data,seq_length_pred):
    
    tamano = int(len(data))
    X,Y_true = [],[]
    # se recorre todo los datos de test
    for j in range(tamano):
        traj = data[j]
        
        lon = traj.shape[0]-seq_length_obs-seq_length_pred
        #lon=data.shape[0]-seq_length_obs-seq_length_pred
        for i in range(0,lon+1):
            a = traj[i:(i +seq_length_obs ), :]
            
            X.append(a)
            # esta parte tiene tanto la parte observada como 
            b = traj[i: (i+seq_length_obs+seq_length_pred), :]
        
            Y_true.append(b)
    return np.array(X),np.array(Y_true)

In [ ]:
# Esta funcion predice posiciones pred normalizados

def sample(datos, seq_length_obs, seq_length_pred):
    
    X,Y_true = secuencia_pred(seq_length_obs,datos,seq_length_pred)
    total_error = 0.0
    total_final = 0.0
    todo=[]
    
    for i in range(len(X)):
        traj_obs = X[i]
        traj_pred = X[i]
    
        #print(Y_true[i])
        
        for j in range(seq_length_pred):
            traj_obsr = numpy.reshape(traj_obs, (1,traj_obs.shape[0],traj_obs.shape[1]) )
            #print(traj_obsr)
            predict = modelxy.predict(traj_obsr)
            next_point = predict
            traj_obs = np.concatenate((traj_obs[1:len(traj_obs)],next_point),axis=0)
            traj_pred = np.concatenate((traj_pred,next_point),axis=0)
            
        diff=Y_true[i][seq_length_obs:]-traj_pred[seq_length_obs:]
        diff=diff**2
        diff=np.sqrt(np.sum(diff,axis=1))
        todo.append(diff)
    
        total_error +=  error_promedio(traj_pred , Y_true[i], seq_length_obs)
        total_final += error_desplazamiento_final(traj_pred, Y_true[i])
        
        
    #print(todo)
    error_ade_modelo = total_error/len(X)
    error_fde_modelo = total_final/len(X)
    
    ade=[t for o in todo for t in o]
    
    print('---------Error--------')
    print('ADE')
    print(np.mean(ade))

    print(error_ade_modelo)
    print('FDE')
    print(error_fde_modelo)
        

In [ ]:
# Esta funcion hace la prediccion en coordenadas pixel
def sample_en_pixeles(datos, seq_length_obs, seq_length_pred):
    
    X,Y_true = secuencia_pred(seq_length_obs,datos,seq_length_pred)
    total_error = 0.0
    total_final = 0.0
    todo = []
    
    for i in range(len(X)):
        traj_obs = X[i]
        traj_pred = X[i]
    
        #print(Y_true[i])
        
        for j in range(seq_length_pred):
            traj_obsr = numpy.reshape(traj_obs, (1,traj_obs.shape[0],traj_obs.shape[1]) )
            #print(traj_obsr)
            predict = modelxy.predict(traj_obsr)
            next_point = predict
            traj_obs = np.concatenate((traj_obs[1:len(traj_obs)],next_point),axis=0)
            traj_pred = np.concatenate((traj_pred,next_point),axis=0)
        
        traj_pre =  np.column_stack((768*traj_pred[:,0],576*traj_pred[:,1]))
        traj_tr = np.column_stack((768*Y_true[i][:,0],576*Y_true[i][:,1]))
          
        diff = traj_pre[seq_length_obs:]-traj_tr[seq_length_obs:]
        diff = diff**2
        diff = np.sqrt(np.sum(diff,axis=1))
        todo.append(diff)
        
        total_error += error_promedio(traj_pre , traj_tr, seq_length_obs)
        total_final += error_desplazamiento_final(traj_pre, traj_tr)
             
    error_modelo = total_error/len(X)
    error_fde_modelo = total_final/len(X)
    
    ade=[t for o in todo for t in o]
    
    print('---------Error--------')
    print('ADE')
    print(np.mean(ade))
    
    print('Error promedio')
    print(error_modelo)
    print('FDE')
    print(error_fde_modelo)     

In [ ]:
sample(datos_prueb,8,4)

In [ ]:
sample_en_pixeles(test5,8,4)

In [ ]:
todo=[0.011135076322301925,0.01779901295573725,0.010485664737981631,0.013785845624867498,0.03644201283933346]

In [ ]:
np.mean(todo)

## Las primeras 8 posiciones de todos los peatones 

In [ ]:
def secuencia_x_persona(seq_length_obs,data,seq_length_pred):
    tamano = int(len(data))
    X,Y_true = [],[]
    for j in range(tamano):
        traj = data[j]
        X.append(traj[0:seq_length_obs,:])
        Y_true.append(traj[0:seq_length_obs+seq_length_pred,:])
    return np.array(X),np.array(Y_true)

In [ ]:
def sample_x_persona_normalizado(datos, seq_length_obs, seq_length_pred):
    
    X,Y_true = secuencia_x_persona(seq_length_obs,datos,seq_length_pred)
    total_error = 0.0
    plt.figure(figsize=(18,15)) 
    color_names = ["r","crimson" ,"g", "b","c","m","y","lightcoral", "peachpuff","grey","springgreen" ,"fuchsia","violet","teal","seagreen","lime","yellow","coral","aquamarine","hotpink"]

    todo = []
    for i in range(len(X)):
        traj_obs = X[i]
        traj_pred = X[i]
    
        for j in range(seq_length_pred):
            traj_obsr = numpy.reshape(traj_obs, (1,traj_obs.shape[0],traj_obs.shape[1]) )
            #print(traj_obsr)
            predict = modelxy.predict(traj_obsr)
            next_point = predict
          
            traj_obs = np.concatenate((traj_obs[1:len(traj_obs)],next_point),axis=0)
            traj_pred = np.concatenate((traj_pred,next_point),axis=0)
        
        diff = Y_true[i][seq_length_obs:]-traj_pred[seq_length_obs:]
        diff = diff**2
        diff = np.sqrt(np.sum(diff,axis=1))
        
        todo.append(diff)
        error_prom = error_promedio(traj_pred ,Y_true[i], seq_length_obs)
        FDE = error_desplazamiento_final(traj_pred , Y_true[i])
        

        print("TRAYECTORIA PREDICHA DEL PEATON ",i+1)
        print(traj_pred)
        print("TRAYECTORIA VERDADERA  ")
        print(Y_true[i])
        print("EL ERROR PROMEDIO DE LA TRAYECTORIA PREDICHA ES = {:.3f} ".format(error_prom))
        print("ERROR DE DESPLAZAMIENTO FINAL ES ={:.3f}".format(FDE))
                         
        #cpu = random.choice(range(17))
                         
        subplot(1,1,1)
        
        
        #predicha=plt.plot(X[i][:,0],X[i][:,1],'*--',color=color_names[cpu])
        predicha=plt.plot(Y_true[i][:,0],Y_true[i][:,1],'*--',color=color_names[19-i])
        predicha=plt.plot(traj_pred[seq_length_obs-1:(seq_length_obs+seq_length_pred),0],
                                  traj_pred[seq_length_obs-1:(seq_length_obs+seq_length_pred),1],'o--',color=color_names[i])
        plt.title("Cuatro posiciones predichas con LTM-X-Y") 
        plt.xlabel('Coordenada x')
        plt.ylabel('Coordenada y')
        
        
        total_error += error_prom
    ade=[t for o in todo for t in o]
    error_modelo = total_error/len(X)
    print("EL ERROR PROMEDIO DE TODA LA PREDICCIÓN")
    print(error_modelo)
    print("Error ade")
    print(np.mean(ade))
    plt.savefig("4predichas.pdf")
    plt.show()

In [ ]:
sample_x_persona_normalizado(datos,8,4)

In [ ]:
def sample_x_persona_pixeles(datos, seq_length_obs, seq_length_pred):
    
    X,Y_true = secuencia_x_persona(seq_length_obs,datos,seq_length_pred)
    total_error = 0.0
    plt.figure(figsize=(18,15))  
    color_names = ["r", "g", "b","c","m","y", "peachpuff","grey", "fuchsia","violet",
                   "teal","seagreen","lime","yellow","coral","aquamarine","hotpink"]
    todo=[]
    for i in range(len(X)):
        traj_obs = X[i]
        traj_pred = X[i]
    
        for j in range(seq_length_pred):
            traj_obsr = numpy.reshape(traj_obs, (1,traj_obs.shape[0],traj_obs.shape[1]) )
            #print(traj_obsr)
            predict = modelxy.predict(traj_obsr)
            next_point = predict
          
            traj_obs = np.concatenate((traj_obs[1:len(traj_obs)],next_point),axis=0)
            traj_pred = np.concatenate((traj_pred,next_point),axis=0)
    
    ss
        traj_pre =  np.column_stack((768*traj_pred[:,0],576*traj_pred[:,1]))
        traj_tr = np.column_stack((768*Y_true[i][:,0],576*Y_true[i][:,1]))
       
        
        diff = traj_tr[seq_length_obs:]-traj_pre[seq_length_obs:]
        diff = diff**2
        diff = np.sqrt(np.sum(diff,axis=1))
        
        todo.append(diff)
        error_prom = error_promedio(traj_pre , traj_tr, seq_length_obs)
        FDE = error_desplazamiento_final(traj_pre , traj_tr)
        
    

        print("TRAYECTORIA PREDICHA DEL PEATON ",i+1)
        print(traj_pre)
        print("TRAYECTORIA VERDADERA  ")
        print(traj_tr)
        print("EL ERROR PROMEDIO DE LA TRAYECTORIA PREDICHA ES = {:.3f} ".format(error_prom))
        print("ERROR DE DESPLAZAMIENTO FINAL ES ={:.3f}".format(FDE))
                         
        cpu = random.choice(range(17))
                         
        subplot(1,1,1)
        
        predicha=plt.plot(traj_tr[:,0],traj_tr[:,1],'*--',color=color_names[cpu])                
        predicha=plt.plot(traj_pre[seq_length_obs:(seq_length_obs+seq_length_pred),0],
                                  traj_pre[seq_length_obs:(seq_length_obs+seq_length_pred),1],'o--',color=color_names[cpu])
        
        plt.title("Cuatro posiciones predichas con LSTM-DX-DY") 
        plt.xlabel('Coordenada x')
        plt.ylabel('Coordenada y')
        
        
        total_error += error_prom
    error_modelo = total_error/len(X)
    ade=[t for o in todo for t in o]
    print("EL ERROR PROMEDIO DE TODA LA PREDICCIÓN")
    print(error_modelo)
    print('Error ADE')
    print(np.mean(ade))
    #plt.savefig("trayectoriasdxdy_pixel.jpg")
    plt.show()
    


In [ ]:
sample_x_persona_pixeles(datos,8,4)

## Trayectorias para la evaluacion cualitativa

In [13]:
cruce = []
cruce.append(datos[3][252:264,:])
cruce.append(datos[4][135:147,:])

paralelos = []
paralelos.append(datos[2][1:13])
paralelos.append(datos[3][20:32])

inverso = []
inverso.append(datos[4][167:179])
inverso.append(datos[6][15:27])

In [14]:
cruce[0]

array([[0.30011341, 0.40633819],
       [0.30590104, 0.40479427],
       [0.31172969, 0.40319566],
       [0.3176612 , 0.40145833],
       [0.32378333, 0.39946319],
       [0.33016445, 0.39711771],
       [0.3368043 , 0.39442187],
       [0.34361732, 0.39148264],
       [0.35047604, 0.38845035],
       [0.35723346, 0.3854776 ],
       [0.36374049, 0.38270017],
       [0.36989193, 0.38021059]])

In [ ]:
# Esta funcion hace la prediccion en coordenadas pixel
def sample_en_pixeles_cualitativamente(datos, seq_length_obs, seq_length_pred):
    
    
    X,Y_true = secuencia_pred(seq_length_obs,datos,seq_length_pred)
    total_error = 0.0
    total_final = 0.0
    todo = []
    trayectoria = []
    verdadero= []
    
    for i in range(len(X)):
        traj_obs = X[i]
        traj_pred = X[i]
        
        for j in range(seq_length_pred):
            traj_obsr = numpy.reshape(traj_obs, (1,traj_obs.shape[0],traj_obs.shape[1]) )
            #print(traj_obsr)
            predict = modelxy.predict(traj_obsr)
            next_point = predict
            traj_obs = np.concatenate((traj_obs[1:len(traj_obs)], next_point), axis = 0)
            traj_pred = np.concatenate((traj_pred, next_point), axis = 0)
        
        traj_pre =  np.column_stack((768*traj_pred[:,0],576*traj_pred[:,1]))
        traj_tr = np.column_stack((768*Y_true[i][:,0],576*Y_true[i][:,1]))
       
        trayectoria.append(traj_pre)
        verdadero.append(traj_tr)
        #SE CALCULA LA METRICA ADE 
        diff = traj_pre[seq_length_obs:]-traj_tr[seq_length_obs:]
        diff = diff**2
        diff = np.sqrt(np.sum(diff,axis=1))
        todo.append(diff)
        
        total_error += error_promedio(traj_pre , traj_tr, seq_length_obs)
        total_final += error_desplazamiento_final(traj_pre, traj_tr)
            
    error_modelo = total_error/len(X)
    error_fde_modelo = total_final/len(X)
    
    ade = [t for o in todo for t in o]
    
    print('---------Error--------')
    print('ADE')
    print(np.mean(ade))
    
    print('Error promedio')
    print(error_modelo)
    print('FDE')
    print(error_fde_modelo)
    return trayectoria, verdadero

In [ ]:
#sample_en_pixeles(cruce,8,4)
p,v = sample_en_pixeles_cualitativamente(paralelos,8,4)

In [ ]:
color_names = ["r", "g", "b","c","m","y", "peachpuff","grey", "fuchsia","violet",
                   "teal","seagreen","lime","yellow","coral","aquamarine","hotpink"]


plot(p[0][0:8,0],p[0][0:8,1],'*--',color= color_names[2],label = 'Observado')
plot(p[0][7:,0],p[0][7:,1],'-',color=color_names[2],label='Predicho')
plot(v[0][7:,0],v[0][7:,1],'--',color=color_names[4],label='GT')

plot(p[1][0:8,0],p[1][0:8,1],'*--',color= color_names[2])
plot(p[1][7:,0],p[1][7:,1],'-',color=color_names[2])
plot(v[1][7:,0],v[1][7:,1],'--',color=color_names[4])

#plt.show()
plt.legend()
plt.savefig("paralelos_absoluto.jpg")
#plt.show()